In [1]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage

## Building out the Dataframes

In [15]:
SQL = '''SELECT * from Transfers247 '''
df_247 = (fx.connDBAndReturnDF(SQL)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

,ID,PlayerName,Year,StandardizedPosition,KeyPositionGroup
247_IDYR,,,,,
paulthurston_nebraska_2012,paulthurston_nebraska,paulthurston,2012,OL,1
gregmcmullen_nebraska_2012,gregmcmullen_nebraska,gregmcmullen,2012,DE,2
jordanwesterkamp_nebraska_2012,jordanwesterkamp_nebraska,jordanwesterkamp,2012,WR,1
imanicross_nebraska_2012,imanicross_nebraska,imanicross,2012,RB,1
tommyarmstrong_nebraska_2012,tommyarmstrong_nebraska,tommyarmstrong,2012,QB,1
...,...,...,...,...,...
dondrellharris_troy_2013,dondrellharris_troy,dondrellharris,2013,QB,1
jordanchunn_troy_2013,jordanchunn_troy,jordanchunn,2013,FB,1
bryanslater_troy_2013,bryanslater_troy,bryanslater,2013,DE,2


In [16]:
SQL = '''SELECT * from UnlinkedNCAA '''
df_ncaa = (fx.connDBAndReturnDF(SQL)).set_index('ID')
df_ncaa.index.name = 'NCAA_ID'
df_ncaa

,PlayerName,Year,StandardizedPosition,KeyPositionGroup
NCAA_ID,,,,
aadarkwah_massachusetts,aadarkwah,2017,WR,1
aadreekisconner_mississippistate,aadreekisconner,2019,DB,2
aarenvaughns_utahstate,aarenvaughns,2017,WR,1
aarionharvey_centralmichigan,aarionharvey,2020,OL,1
aaronabbott_easternmichigan,aaronabbott,2015,DB,2
...,...,...,...,...
zionjohnson_bostoncollege,zionjohnson,2019,OL,1
zionjohnson_louisville,zionjohnson,2019,OL,1
zionkelly_airforce,zionkelly,2020,DB,2


In [17]:
indexer = recordlinkage.Index()
indexer = recordlinkage.BlockIndex(on=['KeyPositionGroup'])


In [18]:
pairs = indexer.index(df_ncaa, df_247)

In [19]:
from functions import YearOther

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
sumFields.append('StandardizedPosition')
c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
sumFields.append('KeyPositionGroup')
c.add(YearOther('Year', 'Year', label='Year'))
sumFields.append('Year')


In [20]:
features = c.compute(pairs, df_ncaa, df_247)

In [21]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

PlayerName  \
NCAA_ID                    247_IDYR                                               
aadarkwah_massachusetts    paulthurston_nebraska_2012                  0.166667   
                           jordanwesterkamp_nebraska_2012              0.250000   
                           imanicross_nebraska_2012                    0.100000   
                           tommyarmstrong_nebraska_2012                0.142857   
                           alonzomoore_nebraska_2012                   0.090909   
...                                                                         ...   
zekezaragoza_oklahomastate luismartinez_georgiasouthern_2016           0.166667   
                           christianjohnstone_appalachianstate_2019    0.111111   
                           gavinpatterson_southalabama_2015            0.071429   
                           massimobiscardi_coastalcarolina_2018        0.066667   
                           jackmartin_troy_2019                        0.166667   

                                                                     StandardizedPosition  \
NCAA_ID                    247_IDYR                                                         
aadarkwah_massachusetts    paulthurston_nebraska_2012                                   0   
                           jordanwesterkamp_nebraska_2012                               1   
                           imanicross_nebraska_2012                                     0   
                           tommyarmstrong_nebraska_2012                                 0   
                           alonzomoore_nebraska_2012                                    1   
...                                                                                   ...   
zekezaragoza_oklahomastate luismartinez_georgiasouthern_2016                            0   
                           christianjohnstone_appalachianstate_2019                     1   
                           gavinpatterson_southalabama_2015                             0   
                           massimobiscardi_coastalcarolina_2018                         0   
                           jackmartin_troy_2019                                         0   

                                                                     KeyPositionGroup  \
NCAA_ID                    247_IDYR                                                     
aadarkwah_massachusetts    paulthurston_nebraska_2012                               1   
                           jordanwesterkamp_nebraska_2012                           1   
                           imanicross_nebraska_2012                                 1   
                           tommyarmstrong_nebraska_2012                             1   
                           alonzomoore_nebraska_2012                                1   
...                                                                               ...   
zekezaragoza_oklahomastate luismartinez_georgiasouthern_2016                        1   
                           christianjohnstone_appalachianstate_2019                 1   
                           gavinpatterson_southalabama_2015                         1   
                           massimobiscardi_coastalcarolina_2018                     1   
                           jackmartin_troy_2019                                     1   

                                                                     Year  \
NCAA_ID                    247_IDYR                                         
aadarkwah_massachusetts    paulthurston_nebraska_2012                 1.0   
                           jordanwesterkamp_nebraska_2012             1.0   
                           imanicross_nebraska_2012                   1.0   
                           tommyarmstrong_nebraska_2012               1.0   
                           alonzomoore_nebraska_2012                  1.0   
...                                                                   ...   
zekez

In [25]:
#features.insert(0, 'sourceID', features.index.get_level_values(0))
#features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['sum'].idxmax()]
    if (data['sum'] > .949):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsNCAATransfer.csv")

## Everything below wrote out Fuzzy Match above a specific threshold (.983)
> Next I have to review the next 500 records for matches

In [13]:
links = dfFinal.values.tolist()
links[0]

['aaronbennett_kansasstate',
 'aaronbennett_texastech_2013',
 1.0,
 1,
 1,
 1.0,
 1.0]

In [14]:
for record in links:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 5, 3, record[5], 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence, Transfer)
        VALUES (?,?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()

## Below is to create and save annotations

In [ ]:
fuzzyMI = pd.MultiIndex.from_frame(dfFinal)
recordlinkage.write_annotation_file(
    "../Annotations/Annotations/annotation_ACTransfersSept.json",
    fuzzyMI[0:150],
    df_ac,
    df_247,
    dataset_a_name="All Conference",
    dataset_b_name="Master"
)